# Grup Mazwell Autonomous Park Project ELE495

**In order to get the results we claim, you need to follow the instructions**

1) you need to import necessary packages, number detection model and initialize camera

In [ ]:
import time
import serial
import torch
import torchvision
import cv2
import numpy as np
import torchvision.transforms as transforms
import torch.nn.functional as F
import PIL.Image
import traitlets
from IPython.display import display
import ipywidgets
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg, Robot
import serial

print("Model Kurumu")
model_num_detect = torchvision.models.alexnet(pretrained=False)
model_num_detect.classifier[6] = torch.nn.Linear(model_num_detect.classifier[6].in_features, 10)
print("Model Yükleme")
model_num_detect.load_state_dict(torch.load('number_detect.pth'))
print("GPU Aktarma")
device = torch.device('cuda')
model_num_detect = model_num_detect.to(device)

mean_num = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean_num, stdev)

def preprocess_num(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

robot = Robot()
camera = Camera.instance(width=224, height=224)

image = ipywidgets.widgets.Image(format='jpeg', width=224, height=224)
n1 = ipywidgets.widgets.FloatSlider(description='n1', min=0.0, max=1.0, orientation='vertical')
n2 = ipywidgets.widgets.FloatSlider(description='n2', min=0.0, max=1.0, orientation='vertical')
n3 = ipywidgets.widgets.FloatSlider(description='n3', min=0.0, max=1.0, orientation='vertical')
n4 = ipywidgets.widgets.FloatSlider(description='n4', min=0.0, max=1.0, orientation='vertical')
n5 = ipywidgets.widgets.FloatSlider(description='n5', min=0.0, max=1.0, orientation='vertical')
n6 = ipywidgets.widgets.FloatSlider(description='n6', min=0.0, max=1.0, orientation='vertical')
n7 = ipywidgets.widgets.FloatSlider(description='n7', min=0.0, max=1.0, orientation='vertical')
n8 = ipywidgets.widgets.FloatSlider(description='n8', min=0.0, max=1.0, orientation='vertical')
n9 = ipywidgets.widgets.FloatSlider(description='n9', min=0.0, max=1.0, orientation='vertical')
n10 = ipywidgets.widgets.FloatSlider(description='n10', min=0.0, max=1.0, orientation='vertical')
ileri = widgets.FloatSlider(description='ileri', min=0.0, max=1.0, orientation='vertical')
left = ipywidgets.widgets.FloatSlider(description='Turn Left', min=0.0, max=1.0, orientation='vertical')
right = ipywidgets.widgets.FloatSlider(description='Turn Right', min=0.0, max=1.0, orientation='vertical')
park = ipywidgets.widgets.FloatSlider(description='Park', min=0.0, max=1.0, orientation='vertical')
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(ipywidgets.widgets.VBox([widgets.HBox([image])]))
# display(ipywidgets.widgets.VBox([widgets.HBox([image, n1, n2, n3, n4, ileri, left, right, park])]))
print("Kamera")
def update(change):
    global robot
    x = change['new'] 
    x = preprocess_num(x)
    y = model_num_detect(x)
    y = F.softmax(y, dim=1)
    num1 = float(y.flatten()[0])
    num2 = float(y.flatten()[2])
    num3 = float(y.flatten()[3])
    num4 = float(y.flatten()[4])
    num5 = float(y.flatten()[5])
    num6 = float(y.flatten()[6])
    num7 = float(y.flatten()[7])
    num8 = float(y.flatten()[8])
    num9 = float(y.flatten()[9])
    num10 = float(y.flatten()[1])
    n1.value = num1
    n2.value = num2 
    n3.value = num3
    n4.value = num4
    n5.value = num5
    n6.value = num6 
    n7.value = num7
    n8.value = num8
    n9.value = num9
    n10.value = num10
update({'new': camera.value}) 

2) Next step is to import road following model and set the speed parameters

In [ ]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value = 0.08, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.01, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.05, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')


x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')
print("Model Kurumu")
model_road = torchvision.models.resnet18(pretrained=False)
model_road.fc = torch.nn.Linear(512, 2)
print("Model Yükleme")
model_road.load_state_dict(torch.load('yilan.pth'))
print("GPU aktarım")
device = torch.device('cuda')
model_road = model_road.to(device)
model_road = model_road.eval().half()

mean_road = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess_road(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean_road[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

angle = 0.0
angle_last = 0.0
def execute(change):
    global angle, angle_last
    image = change['new']
    xy = model_road(preprocess_road(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value  
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)

3) In the next three cells we are going to initialize parking functions.

In [ ]:
def parki(num = 7):
    yon = 0
    park.value = 0
    while not yon:
        if num == 1:
            check = n1.value
            left_value = left.value
            right_value = right.value
            stop_value = park.value
        elif num == 2:
            check = n2.value
            left_value = left.value
            right_value = right.value
            stop_value = park.value
        elif num == 3:
            check = n3.value
            left_value = left.value
            right_value = right.value
            stop_value = park.value
        elif num == 4:
            check = n4.value
            left_value = left.value
            right_value = right.value
            stop_value = park.value
        elif num == 5:
            check = n5.value
            left_value = left.value
            right_value = right.value
            stop_value = park.value
        elif num == 6:
            check = n6.value
            left_value = left.value
            right_value = right.value
            stop_value = park.value
        elif num == 7:
            check = n7.value
            left_value = left.value
            right_value = right.value
            stop_value = park.value
        elif num == 8:
            check = n8.value
            left_value = left.value
            right_value = right.value
            stop_value = park.value
        elif num == 9:
            check = n9.value
            left_value = left.value
            right_value = right.value
            stop_value = park.value
        elif num == 10:
            check = n10.value
            left_value = left.value
            right_value = right.value
            stop_value = park.value
        if check > left_value and check > right_value:
            robot.forward(0.1)
            time.sleep(0.1)
            robot.stop()
            time.sleep(0.1)
        if left_value > check and left_value > right_value:
            robot.left(0.09)
            time.sleep(0.1)
            robot.stop()
            robot.forward(0.09)
            time.sleep(0.1)
            robot.stop()
            time.sleep(0.1)
        if right_value > check and right_value > left_value:
            robot.right(0.09)
            time.sleep(0.1)
            robot.stop()
            robot.forward(0.09)
            time.sleep(0.1)
            robot.stop()
            time.sleep(0.1)
        if stop_value > 0.86:
            print(f"Stop değeri {stop_value}")
            yon = 1
            robot.stop()
            time.sleep(0.1)
    return 1

In [ ]:
def pre_def_f(deger, num = 7):
    if num == 1:
        if deger == 0:
            camera.observe(update_1, names='value')
            pass
        elif deger == 1:
            camera.unobserve(update_1, names='value')
            pass
        elif deger == 2:
            return n1.value
    elif num == 2:
        if deger == 0:
            camera.observe(update_2, names='value')
            pass
        elif deger == 1:
            camera.unobserve(update_2, names='value')
            pass
        elif deger == 2:
            return n2.value
    elif num == 3:
        if deger == 0:
            camera.observe(update_3, names='value')
            pass
        elif deger == 1:
            camera.unobserve(update_3, names='value')
            pass
        elif deger == 2:
            return n3.value
    elif num == 4:
        if deger == 0:
            camera.observe(update_4, names='value')
            pass
        elif deger == 1:
            camera.unobserve(update_4, names='value')
            pass
        elif deger == 2:
            return n4.value
    elif num == 5:
        if deger == 0:
            camera.observe(update_5, names='value')
            pass
        elif deger == 1:
            camera.unobserve(update_5, names='value')
            pass
        elif deger == 2:
            return n5.value
    elif num == 6:
        if deger == 0:
            camera.observe(update_6, names='value')
            pass
        elif deger == 1:
            camera.unobserve(update_6, names='value')
            pass
        elif deger == 2:
            return n6.value
    elif num == 7:
        if deger == 0:
            camera.observe(update_7, names='value')
            pass
        elif deger == 1:
            camera.unobserve(update_7, names='value')
            pass
        elif deger == 2:
            return n7.value
    elif num == 8:
        if deger == 0:
            camera.observe(update_8, names='value')
            pass
        elif deger == 1:
            camera.unobserve(update_8, names='value')
            pass
        elif deger == 2:
            return n8.value
    elif num == 9:
        if deger == 0:
            camera.observe(update_9, names='value')
            pass
        elif deger == 1:
            camera.unobserve(update_9, names='value')
            pass
        elif deger == 2:
            return n9.value
    elif num == 10:
        if deger == 0:
            camera.observe(update_10, names='value')
            pass
        elif deger == 1:
            camera.unobserve(update_10, names='value')
            pass
        elif deger == 2:
            return n10.value

In [ ]:
def yol_park(n = 7):
    key = 0
    camera.observe(execute, names='value')
    start_time = time.time()
    elapsed_time = 1
    final_time = 0
    first_time = 0
    m = 0
    while elapsed_time < 102: 
        time.sleep(4.2-m)
        camera.unobserve(execute, names='value')
        time.sleep(0.1) 
        robot.stop()
        camera.observe(update, names='value') 
        girdi = 0
        for a in range(14):
            robot.left(0.1)
            time.sleep(0.3)
            robot.stop()
            if first_time == 0:
                first_time += 1
                m = 1.105
            time.sleep(1.2)
            if pre_def_f(2, n) > 0.8:
                girdi = 1
                camera.unobserve(update, names='value')
                time.sleep(0.1)
                pre_def_f(0, n)
                if parki():
                    key = 1
                    print(key)
                    pre_def_f(1, n)
                    time.sleep(0.01)
                    break
            time.sleep(0.01)
        if girdi == 0:
            camera.unobserve(update, names='value')
        if key == 1:
            return 1
        time.sleep(0.1)
        robot.stop()
        final_time = time.time()
        elapsed_time = final_time - start_time
        camera.observe(execute, names='value')
    if key == 0:
        camera.unobserve(execute, names='value')
        return 0
    time.sleep(0.1) 
    robot.stop()

4) Next step is to make a connection with phone application provided for you in the code section. After making connection you need to insert parking lot number. Then through application you will get it status.

In [ ]:
#def park_alanı(num = 7):

serial_port = serial.Serial(
    port="/dev/ttyTHS1",
    baudrate=9600,
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
)

num = 7


try:
    # Send a simple header
    #serial_port.write("UART Demonstration Program\r\n".encode())
    #serial_port.write("NVIDIA Jetson Nano Developer Kit\r\n".encode())

    while True:

        if serial_port.inWaiting() > 0:
            data = serial_port.read(2)
#             time.sleep(1)
            num = int(data.decode());
            print(num)
            if(num != 0):
                if num == 1:
                    model = num
                    print("Model Kurumu")
                    serial_port.write("7".encode('utf-8'))
                    model_park1 = torchvision.models.alexnet(pretrained=False)
                    model_park1.classifier[6] = torch.nn.Linear(model_park1.classifier[6].in_features, 4)
                    print("Model Yükleme")
                    model_park1.load_state_dict(torch.load('Park1.pth'))
                    print("GPU aktarımı")
                    serial_port.write("8".encode('utf-8'))
                    device = torch.device('cuda')
                    model_park1 = model_park1.to(device)
                    print("Kamera fonksiyonu")
                    serial_port.write("9".encode('utf-8'))
                    def update_1(change):
                        global robot
                        x = change['new'] 
                        x = preprocess_num(x)
                        y = model_park1(x)
                        y = F.softmax(y, dim=1)
                        r = float(y.flatten()[0])
                        p = float(y.flatten()[1])
                        num1 = float(y.flatten()[2])
                        l= float(y.flatten()[3])
                        n1.value = num1
                        right.value = r 
                        left.value = l
                        park.value = p
                    print("Kamera tanımlı")
                elif num == 2:
                    model = num
                    print("Model Kurumu")
                    serial_port.write("7".encode('utf-8'))
                    model_park2 = torchvision.models.alexnet(pretrained=False)
                    model_park2.classifier[6] = torch.nn.Linear(model_park2.classifier[6].in_features, 4)
                    print("Model Yükleme")
                    model_park2.load_state_dict(torch.load('Park2.pth'))
                    print("GPU aktarımı")
                    serial_port.write("8".encode('utf-8'))
                    device = torch.device('cuda')
                    model_park2 = model_park2.to(device)
                    print("Kamera fonksiyonu")
                    serial_port.write("9".encode('utf-8'))
                    def update_2(change):
                        global robot
                        x = change['new'] 
                        x = preprocess_num(x)
                        y = model_park2(x)
                        y = F.softmax(y, dim=1)
                        r = float(y.flatten()[0])
                        p = float(y.flatten()[1])
                        num2 = float(y.flatten()[2])
                        l= float(y.flatten()[3])
                        n2.value = num2
                        right.value = r 
                        left.value = l
                        park.value = p
                    print("Kamera tanımlı")
                    
                elif num == 3:
                    model = num
                    print("Model Kurumu")
                    serial_port.write("7".encode('utf-8'))
                    model_park3 = torchvision.models.alexnet(pretrained=False)
                    model_park3.classifier[6] = torch.nn.Linear(model_park3.classifier[6].in_features, 4)
                    print("Model Yükleme")
                    model_park3.load_state_dict(torch.load('Park3.pth'))
                    print("Model Kurumu")
                    serial_port.write("8".encode('utf-8'))
                    device = torch.device('cuda')
                    model_park3 = model_park3.to(device)
                    print("Kamera fonksiyonu")
                    serial_port.write("9".encode('utf-8'))
                    def update_3(change):
                        global robot
                        x = change['new'] 
                        x = preprocess_num(x)
                        y = model_park3(x)
                        y = F.softmax(y, dim=1)
                        r = float(y.flatten()[0])
                        p = float(y.flatten()[1])
                        num3 = float(y.flatten()[2])
                        l= float(y.flatten()[3])
                        n3.value = num3
                        right.value = r 
                        left.value = l
                        park.value = p
                    print("Kamera tanımlı")
                elif num == 4:
                    model = num
                    print("Model Kurumu")
                    serial_port.write("7".encode('utf-8'))
                    model_park4 = torchvision.models.alexnet(pretrained=False)
                    model_park4.classifier[6] = torch.nn.Linear(model_park4.classifier[6].in_features, 4)
                    print("Model Yükleme")
                    model_park4.load_state_dict(torch.load('Park4.pth'))
                    print("GPU aktarım")
                    serial_port.write("8".encode('utf-8'))
                    device = torch.device('cuda')
                    model_park4 = model_park4.to(device)
                    print("Kamera fonksiyonu")
                    serial_port.write("9".encode('utf-8'))
                    def update_4(change):
                        global robot
                        x = change['new'] 
                        x = preprocess_num(x)
                        y = model_park4(x)
                        y = F.softmax(y, dim=1)
                        r = float(y.flatten()[0])
                        p = float(y.flatten()[1])
                        num4 = float(y.flatten()[2])
                        l= float(y.flatten()[3])
                        n4.value = num4
                        right.value = r 
                        left.value = l
                        park.value = p
                    print("Kamera tanımlı")
                elif num == 5:
                        model = num
                        serial_port.write("0".encode('utf-8'))
                        print("Model Kurumu")
                        serial_port.write("7".encode('utf-8'))
                        model_5 = torchvision.models.alexnet(pretrained=False)
                        model_5.classifier[6] = torch.nn.Linear(model_5.classifier[6].in_features, 4)
                        print("Model Yükleme")
                        model_5.load_state_dict(torch.load('Park5.pth'))
                        print("GPU aktarımı")
                        serial_port.write("8".encode('utf-8'))
                        device = torch.device('cuda')
                        model_5 = model_5.to(device)
                        print("Kamera fonksiyonu")
                        serial_port.write("9".encode('utf-8'))
                        def update_5(change):
                            global robot
                            x = change['new'] 
                            x = preprocess_num(x)
                            y = model_park6(x)
                            y = F.softmax(y, dim=1)
                            r = float(y.flatten()[0])
                            p = float(y.flatten()[1])
                            num5 = float(y.flatten()[2])
                            l= float(y.flatten()[3])
                            n5.value = num5
                            right.value = r 
                            left.value = l
                            park.value = p
                        print("Kamera tanımlı")
                elif num == 6:
                        model = num
                        print("Model Kurumu")
                        serial_port.write("7".encode('utf-8'))
                        model_park6 = torchvision.models.alexnet(pretrained=False)
                        model_park6.classifier[6] = torch.nn.Linear(model_park6.classifier[6].in_features, 4)
                        print("Model Yükleme")
                        model_park6.load_state_dict(torch.load('Park6.pth'))
                        print("GPU aktarımı")
                        serial_port.write("8".encode('utf-8'))
                        device = torch.device('cuda')
                        model_park6 = model_park6.to(device)
                        print("Kamera fonksiyonu")
                        serial_port.write("9".encode('utf-8'))
                        def update_6(change):
                            global robot
                            x = change['new'] 
                            x = preprocess_num(x)
                            y = model_park6(x)
                            y = F.softmax(y, dim=1)
                            r = float(y.flatten()[0])
                            p = float(y.flatten()[1])
                            num6 = float(y.flatten()[2])
                            l= float(y.flatten()[3])
                            n6.value = num6
                            right.value = r 
                            left.value = l
                            park.value = p
                        print("Kamera tanımlı")
                elif num == 7:
                        model = num
                        print("Model Kurumu")
                        serial_port.write("7".encode('utf-8'))
                        model_park7 = torchvision.models.alexnet(pretrained=False)
                        model_park7.classifier[6] = torch.nn.Linear(model_park7.classifier[6].in_features, 4)
                        print("Model Yükleme")
                        model_park7.load_state_dict(torch.load('Park7.pth'))
                        print("GPU aktarımı")
                        serial_port.write("8".encode('utf-8'))
                        device = torch.device('cuda')
                        model_park7 = model_park7.to(device)
                        print("Kamera fonksiyonu")
                        serial_port.write("9".encode('utf-8'))
                        def update_7(change):
                            global robot
                            x = change['new'] 
                            x = preprocess_num(x)
                            y = model_park7(x)
                            y = F.softmax(y, dim=1)
                            r = float(y.flatten()[0])
                            p = float(y.flatten()[1])
                            num7 = float(y.flatten()[2])
                            l= float(y.flatten()[3])
                            n7.value = num7
                            right.value = r 
                            left.value = l
                            park.value = p
                        print("Kamera tanımlı")
                elif num == 8:
                        model = num
                        print("Model Kurumu")
                        serial_port.write("7".encode('utf-8'))
                        model_park8 = torchvision.models.alexnet(pretrained=False)
                        model_park8.classifier[6] = torch.nn.Linear(model_park8.classifier[6].in_features, 4)
                        print("Model Yükleme")
                        model_park8.load_state_dict(torch.load('Park8.pth'))
                        print("GPU aktarımı")
                        serial_port.write("8".encode('utf-8'))
                        device = torch.device('cuda')
                        model_park8 = model_park8.to(device)
                        print("Kamera fonksiyonu")
                        serial_port.write("9".encode('utf-8'))
                        def update_8(change):
                            global robot
                            x = change['new'] 
                            x = preprocess_num(x)
                            y = model_park8(x)
                            y = F.softmax(y, dim=1)
                            r = float(y.flatten()[0])
                            p = float(y.flatten()[1])
                            num8 = float(y.flatten()[2])
                            l= float(y.flatten()[3])
                            n8.value = num8
                            right.value = r 
                            left.value = l
                            park.value = p
                        print("Kamera tanımlı")
                elif num == 9:
                        model = num
                        print("Model Kurumu")
                        serial_port.write("7".encode('utf-8'))
                        model_park9 = torchvision.models.alexnet(pretrained=False)
                        model_park9.classifier[6] = torch.nn.Linear(model_park9.classifier[6].in_features, 4)
                        print("Model Yükleme")
                        model_park9.load_state_dict(torch.load('Park9.pth'))
                        print("GPU aktarımı")
                        serial_port.write("8".encode('utf-8'))
                        device = torch.device('cuda')
                        model_park9 = model_park9.to(device)
                        print("Kamera fonksiyonu")
                        serial_port.write("9".encode('utf-8'))
                        def update_9(change):
                            global robot
                            x = change['new'] 
                            x = preprocess_num(x)
                            y = model_park9(x)
                            y = F.softmax(y, dim=1)
                            r = float(y.flatten()[0])
                            p = float(y.flatten()[1])
                            num9 = float(y.flatten()[2])
                            l= float(y.flatten()[3])
                            n9.value = num9
                            right.value = r 
                            left.value = l
                            park.value = p
                        print("Kamera tanımlı")
                elif num == 10:
                        model = num
                        print("Model Kurumu")
                        serial_port.write("7".encode('utf-8'))
                        model_park10 = torchvision.models.alexnet(pretrained=False)
                        model_park10.classifier[6] = torch.nn.Linear(model_park10.classifier[6].in_features, 4)
                        print("Model Yükleme")
                        model_park10.load_state_dict(torch.load('Park10.pth'))
                        print("GPU aktarımı")
                        serial_port.write("8".encode('utf-8'))
                        device = torch.device('cuda')
                        model_park10 = model_park10.to(device)
                        print("Kamera fonksiyonu")
                        serial_port.write("9".encode('utf-8'))
                        def update_10(change):
                            global robot
                            x = change['new'] 
                            x = preprocess_num(x)
                            y = model_park10(x)
                            y = F.softmax(y, dim=1)
                            r = float(y.flatten()[0])
                            p = float(y.flatten()[1])
                            num10 = float(y.flatten()[2])
                            l= float(y.flatten()[3])
                            n10.value = num10
                            right.value = r 
                            left.value = l
                            park.value = p
                        print("Kamera tanımlı")
            else:
                print("Park başladı")
                serial_port.write("4".encode('utf-8'))
                if yol_park(model):
                    serial_port.write("6".encode('utf-8'))
                else:
                    serial_port.write("3".encode('utf-8'))
                print("bitti")
                
#                 break
    
except KeyboardInterrupt:
    print("Exiting Program")

except Exception as exception_error:
    print("Error occurred. Exiting Program")
    print("Error: " + str(exception_error))

finally:
    serial_port.close()
    pass

5) As a last step, when you done with the code for the sake of the jetson and the camera we are going to stop them

In [ ]:
robot.stop()
camera.stop()

   **Thank you for your time** 